In [6]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import PIL
from random import shuffle
# from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.naive_bayes import GaussianNB

In [7]:
import pickle

In [3]:
with open('dataconvall.pt', 'rb') as f:
    data = pickle.load(f)
    
with open('labelsall.pt', 'rb') as f:
    labels = pickle.load(f)
    
with open('datavalconvall.pt', 'rb') as f:
    dataVal = pickle.load(f)
    
with open('labelsval.pt', 'rb') as f:
    labelsVal = pickle.load(f)

In [ ]:
avedloss, avedacc, avevloss, avevacc = 0,0,0,0
crit = nn.CrossEntropyLoss()

for i in range(5):
    model = GaussianNB()
    print("Fold %d" % (i + 1))
    trainX = []
    trainY = []
    testX = []
    testY = []
    for j in range(5):
        if i != j:
            trainX = trainX + data[j]
            trainY = trainY + labels[j]
        else:
            testX = data[j]
            testY = labels[j]
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
#     print(trainX.shape)
#     print(trainY.shape)
#     print(testX.shape)
#     print(testY.shape)
    
    
    model.fit(trainX,trainY)
    
    train_out = model.predict(trainX)
    test_out = model.predict(testX)
    train_prob_out = model.predict_proba(trainX)
    test_prob_out = model.predict_proba(testX)
    A = torch.from_numpy(np.asarray(train_prob_out,dtype=np.uint8)).type('torch.FloatTensor')
    B = torch.from_numpy(np.asarray(trainY,dtype=np.uint8)).type('torch.LongTensor')
    folddloss = crit(A,B).item()
    A = torch.from_numpy(np.asarray(test_prob_out,dtype=np.uint8)).type('torch.FloatTensor')
    B = torch.from_numpy(np.asarray(testY,dtype=np.uint8)).type('torch.LongTensor')
    foldvloss = crit(A,B).item()
    folddacc = np.mean(train_out == trainY)
    foldvacc = np.mean(test_out == testY)
    print("Fold Train Loss {:.6f} Fold Train Accuracy {:.6f} Fold Test Loss {:.6f} Fold Test Accuracy {:.6f}".format(folddloss,folddacc,foldvloss,foldvacc))
    avedacc += folddacc
    avevacc += foldvacc
    avedloss += folddloss
    avevloss += foldvloss

avedloss /= 5
avedacc /= 5
avevloss /= 5
avevacc /= 5
print("Average Train Loss {:.6f}\nAverage Train Accuracy: {:.6f}\nAverage Test Loss {:.6f}\nAverage Test Accuracy: {:.6f}\n".format(avedloss,avedacc,avevloss,avevacc))

In [ ]:
avedacc /= 5
avevacc /= 5
print("Average Train Accuracy: {:.6f}\nAverage Test Accuracy: {:.6f}\n".format(avedacc,avevacc))

In [ ]:
avedacc, avevacc = 0,0

model = GaussianNB()
# print("Fold %d" % (i + 1))
trainX = data
trainY = labels
trainX = np.array(trainX)
trainY = np.array(trainY)
print(trainX.shape,trainY.shape)

model.fit(trainX,trainY)

train_out = model.predict(trainX)
temp_out = model.predict_proba(trainX)
folddacc = np.mean(train_out == trainY)
print("Fold Train Accuracy {:.6f} ".format(folddacc))
print(train_out)
print(temp_out)

In [ ]:
crit = nn.CrossEntropyLoss()
temp_out = torch.from_numpy(np.asarray(temp_out,dtype=np.uint8)).type('torch.FloatTensor')
trainY = torch.from_numpy(np.asarray(trainY,dtype=np.uint8)).type('torch.LongTensor')
print(temp_out.shape)
print(trainY.shape)
loss = crit(temp_out, trainY)
print(loss.item())

In [ ]:
with open('naivebayesoutprobs.pt', 'wb') as f:
    pickle.dump(temp_out[:,1], f)

In [ ]:
from sklearn.metrics import *
import itertools
import pickle
with open('naivebayesout.pt', 'rb') as f:
    y_pred = pickle.load(f)
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


y_test = np.array([])

for x in labels:
    y_test = np.append(y_test,x)

class_names = ["bad site","good site"]

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()
print("Accuracy: {:.4f}".format(np.mean(y_test==y_pred)))